# Granular Demand Forecast

In [2]:
# libraries
library(tidyverse)
library(lubridate)
library(RcppRoll)
library(RPostgreSQL)
library(tsibble)
library(fable)
library(gridExtra)
library(repr)
library(clipr)
library(config)
library(googlesheets4)

In [3]:
# setting wd
if ( grepl("granular-demand-forecast", getwd(), fixed = TRUE) ) {
    print("current working directory is fine")
} else {
    setwd("./projects/granular-demand-forecast")
    print("changed working directory")
}

[1] "current working directory is fine"


In [5]:
# load functions
source("granular-demand-forecast-functions.R")

### Get Data

In [6]:
# get credentials
dw <- config::get(file="./redshift_credentials.yml")

In [9]:
# connecting to dw
con <- DBI::dbConnect(RPostgreSQL::PostgreSQL(),
                      dbname = dw$dbname,
                      host = dw$host,
                      port = dw$port,
                      user = dw$user,
                      password = dw$password)

In [8]:
# parameters
grouping_parameter <- "size"

In [11]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

ERROR: Error in parse(text = x, srcfile = src): <text>:2:6: unexpected symbol
1: #import ipywidgets as widgets
2: from ipywidgets
        ^


In [ ]:
# data from dw
df_sales           <- dbGetQuery(con, getSQL("sql/rug_sales.sql")) %>% as_tibble()

# data from gsheets
total_spend        <- suppressMessages(read_sheet("https://docs.google.com/spreadsheets/d/1hPxDLt-MyQZpaZRHDSAXG_wMx-s-G5TdcyPYFnUQeAY/edit#gid=999386989",
                                                  range = "Daily Actuals!E41:41",
                                                  col_names=FALSE)) %>% as_tibble()
dates              <- suppressMessages(read_sheet("https://docs.google.com/spreadsheets/d/1hPxDLt-MyQZpaZRHDSAXG_wMx-s-G5TdcyPYFnUQeAY/edit#gid=999386989",
                                           range = "Daily Actuals!E4:4",
                                           col_names=FALSE)) %>% as_tibble()
dates              <- dates[,1:length(total_spend)]
df_spend           <- suppressMessages(bind_cols(as_date(t(dates)),as.numeric(t(total_spend))) %>% as_tibble())
colnames(df_spend) <- suppressMessages(c("date", "total_spend"))
df_spend           <- df_spend %>% mutate(month_begin_date = floor_date(date, 'month'))
df_monthly_spend   <- df_spend %>% 
                          group_by(month_begin_date) %>% 
                          summarise(total_spend = sum(total_spend))

In [9]:
# reading query from SQL file
options(warn=-1)
query_sales <- getSQL("data_pull__rug_sales.sql")
query_spend <- getSQL(paste0("data_pull__spend-by-",grouping_parameter,".sql"))
query_email <- getSQL(paste0("data_pull__email-by-",grouping_parameter,".sql"))
# pull data
df_sales <- dbGetQuery(con, query_sales) %>% as_tibble()
df_spend <- dbGetQuery(con, query_spend) %>% as_tibble()
df_email <- dbGetQuery(con, query_email) %>% as_tibble()
options(warn=0)

In [10]:
# add column for week begin date
df_sales <- df_sales %>% mutate(week_begin_date = floor_date(as_date(date), unit="week"))
df_spend <- df_spend %>% mutate(week_begin_date = floor_date(as_date(date), unit="week"))
df_email <- df_email %>% mutate(week_begin_date = floor_date(as_date(date), unit="week"))

In [36]:
# add column for week begin date
# df_sales <- df_sales %>% mutate(as.name(grouping_parameter) = tolower((!!as.name(grouping_parameter))))
# df_spend <- df_spend %>% mutate(as.name(grouping_parameter) = tolower((!!as.name(grouping_parameter))))
# df_email <- df_email %>% mutate(as.name(grouping_parameter) = tolower((!!as.name(grouping_parameter))))

ERROR: Error in parse(text = x, srcfile = src): <text>:2:61: unexpected '='
1: # add column for week begin date
2: df_sales <- df_sales %>% mutate(as.name(grouping_parameter) =
                                                               ^


### Data Prep

In [37]:
# group sales data
if (grouping_parameter == "size") {
    df_sales_grouped <- groupData(df_sales,
                                  c("date",grouping_parameter))
} else if (grouping_parameter == "product_sub_type") {
    df_sales_grouped <- groupData(df_sales %>% filter(product_type=="system"),
                                  c("date",grouping_parameter))
} else {
    print("choose a valid grouping parameter")
}
# get distinct groups
groups <- df_sales_grouped %>% distinct_at(grouping_parameter)

In [38]:
# creating xreg
# total sales
df_total_sales <- df_sales %>%
    group_by(date) %>%
    summarize(total_quantity = sum(quantity), .groups = 'drop')
# joining df and xreg
df_sales_w_xreg <- df_sales_grouped %>%
    inner_join(df_total_sales, by="date") %>%
    inner_join(df_spend, by=c("date",grouping_parameter)) %>%
    inner_join(df_email, by=c("date",grouping_parameter)) %>%
    mutate(percent_of_total = quantity/total_quantity,
           dow = as.factor(wday(date)))

### Historical Performance (Last 3 Monthly Snapshots)

In [60]:
historic_performance <- tibble() # create empty tibble that we'll fill later
options(warn=-1)
for (i in 1:3) {
    # train set
    df_train <- df_sales_w_xreg %>%
        as_tsibble(index = date, key = as.name(grouping_parameter)) %>%
        fill_gaps(quantity = 0L) %>%
        filter(date<floor_date(as_date(Sys.Date() %m-% months(i)), unit="month"))
    # test set
    df_test <- df_sales_w_xreg %>%
        as_tsibble(index = date, key = as.name(grouping_parameter)) %>%
        fill_gaps(quantity = 0L) %>%
        filter(date>=floor_date(as_date(Sys.Date() %m-% months(i)), unit="month") &
               date<=floor_date(as_date(Sys.Date() %m-% months(i)), unit="month")+27)
    # fit model
    fit <- df_train %>%
        model(arima = ARIMA(percent_of_total ~
                            total_quantity +
                            total_spend +
                            fb_spend +
                            google_spend +
                            pinterest_spend +
                            bing_spend +
                            #emails_sent_promo +
                            #emails_sent_content +
                            trend() +
                            #season(period = "1 week") +
                            dow)
    # create forecast
    fc <- fit %>%
        forecast(new_data = df_test)
    # accuracy measure
    historic_performance <- historic_performance %>% rbind(fc %>% accuracy(data = df_test) %>% select((!!as.name(grouping_parameter)), MPE, MAPE))    
}
historic_performance <- historic_performance %>% group_by_at(grouping_parameter) %>% summarize(MPE = mean(MPE), MAPE = mean(MAPE), .groups = 'drop')
options(warn=0)

In [65]:
# set parameters according to grouping parameter defined earlier
if (grouping_parameter == "size") {
    # define order
    group_order <- c("2x3","2.5x7","2.5x10","3x5","5x7","6x9","8x10","9x12",'6" round','8" round',"doormat")
    # order according to how its orders in the gsheet
    groups <- groups %>% arrange(match(size, c("2x3","2.5x7","2.5x10","3x5","5x7","6x9","8x10","9x12",'6" round','8" round',"doormat")))
    # set start row
    start_row <- 216
    # set increment
    next_row_distance <- 4
} else if (grouping_parameter == "product_sub_type") {
    # define order
    group_order <- c("cushioned system","classic system")
    # order according to how its orders in the gsheet
    groups <- groups %>% arrange(match(product_sub_type, c("cushioned system","classic system")))
    # set start row
    start_row <- 259
    # set increment
    next_row_distance <- 1
} else {
    print("choose a valid grouping parameter")
}

In [71]:
historic_performance %>%
                    arrange(match((!!as.name(grouping_parameter)), group_order)) %>% mutate(MAPE = MAPE/100, MPE = MPE/100) %>% select(size,MAPE) %>% slice(1)

size,MAPE
<chr>,<dbl>
2.5x7,0.07505093


In [72]:
# loop through to populate gsheet w historic performance
selected_row <- 2
for (i in 1:nrow(groups)) {
    range_write("https://docs.google.com/spreadsheets/d/16XnLir5LtpBBqN4_AgueR5RlUqQo2d_w7s_RizbsM-o/edit?ts=6089979a#gid=570774470",
                data=historic_performance %>%
                    arrange(match((!!as.name(grouping_parameter)), group_order)) %>% mutate(MAPE = MAPE/100, MPE = MPE/100) %>% select(size,MAPE) %>% slice(i),
                range=paste0("Forecast Outputs - Raw!A",selected_row),
                col_names=FALSE,
                reformat=FALSE)
    selected_row <- selected_row+1
}

Auto-refreshing stale OAuth token.

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Raw"

Editing "Daily Product Breakdown Forecast -W

### Forecast

In [73]:
options(warn=-1)
# training set is all data to date
df_train <- df_sales_w_xreg %>%
    as_tsibble(index = date, key = as.name(grouping_parameter)) %>%
    fill_gaps(quantity = 0L)
# test set is from inputs from gsheet
df_gsheet <- read_sheet("https://docs.google.com/spreadsheets/d/16XnLir5LtpBBqN4_AgueR5RlUqQo2d_w7s_RizbsM-o/edit?ts=6089979a#gid=570774470",
                        range = "Ad Spend Inputs!A22:I62",
                        col_names=TRUE) %>% 
    as_tibble() %>%
    filter(category == 'forecast') %>% 
    rename(total_quantity = total_rugs,
           total_spend = projected_total_spend,
           fb_spend = projected_fb_spend,
           google_spend = projected_google_spend,
           pinterest_spend = projected_pinterest_spend,
           bing_spend = projected_bing_spend) %>%
    mutate(date=as_date(date))


df_gsheet$k <- 1
groups$k <- 1
df_test <- df_gsheet %>% # cross join to duplicate gsheet inputs for each size
    inner_join(groups, by='k') %>%
    select(-c(k,week_begin_date)) %>%
    as_tsibble(index = date, key = as.name(grouping_parameter)) %>%
    mutate(dow = as.factor(wday(date)))

Reading from "Daily Product Breakdown Forecast -WIP"

Range "'Ad Spend Inputs'!A22:I62"



In [92]:
# fit model
fit <- df_train %>%
    model(arima = ARIMA(percent_of_total ~
                        total_quantity +
                        total_spend +
                        fb_spend +
                        google_spend +
                        pinterest_spend +
                        bing_spend +
                        #emails_sent_promo +
                        #emails_sent_content +
                        trend() +
                        dow))
options(warn=0)

Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”
Warning message in sqrt(diag(best$var.coef)):
“NaNs produced”


In [94]:
# forecast using new data
fc <- fit %>%
    forecast(new_data = df_test)

In [91]:
fc %>%
        arrange(date) %>%
        as_tibble() %>%
        filter((!!as.name(grouping_parameter))=='5x7') %>%
        select(.mean) %>%
        t() %>%
        as_tibble()

size,.model,date,percent_of_total,.mean,total_quantity,category,total_spend,fb_spend,google_spend,bing_spend,pinterest_spend,dow
<chr>,<chr>,<date>,<dist>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5x7,arima,2021-05-27,"N(0.17, 0.00033)",0.1661955,4642.859,forecast,204738.6,117448.9,27684.39,2348.978,16778.42,5
5x7,ets,2021-05-27,"N(0.17, 0.00023)",0.1744811,4642.859,forecast,204738.6,117448.9,27684.39,2348.978,16778.42,5
5x7,ensemble,2021-05-27,"N(0.17, 0.00027)",0.1703383,4642.859,forecast,204738.6,117448.9,27684.39,2348.978,16778.42,5
5x7,arima,2021-05-28,"N(0.17, 0.00039)",0.1653018,10891.255,forecast,480277.5,275512.6,64942.26,5510.253,39358.95,6
5x7,ets,2021-05-28,"N(0.17, 0.00027)",0.1744811,10891.255,forecast,480277.5,275512.6,64942.26,5510.253,39358.95,6
5x7,ensemble,2021-05-28,"N(0.17, 0.00032)",0.1698914,10891.255,forecast,480277.5,275512.6,64942.26,5510.253,39358.95,6
5x7,arima,2021-05-29,"N(0.16, 0.00042)",0.1627337,11120.340,forecast,490379.6,281307.7,66308.25,5626.154,40186.82,7
5x7,ets,2021-05-29,"N(0.17, 0.00031)",0.1744811,11120.340,forecast,490379.6,281307.7,66308.25,5626.154,40186.82,7
5x7,ensemble,2021-05-29,"N(0.17, 0.00036)",0.1686074,11120.340,forecast,490379.6,281307.7,66308.25,5626.154,40186.82,7


In [89]:
# loop through to populate gsheet
selected_row <- 2 #start_row
for (i in 1:nrow(groups)) {
    s <- groups %>% select(as.name(grouping_parameter)) %>% slice(i) %>% as.character()
    forecasted_rugs <- fc %>%
        arrange(date) %>%
        as_tibble() %>%
        filter((!!as.name(grouping_parameter))==s) %>%
        select(.mean) %>%
        t() %>%
        as_tibble()
    range_write("https://docs.google.com/spreadsheets/d/16XnLir5LtpBBqN4_AgueR5RlUqQo2d_w7s_RizbsM-o/edit?ts=6089979a#gid=570774470",
            data=forecasted_rugs,
            range=paste0("Forecast Outputs - Predictions!A",selected_row,":AC",selected_row),
            col_names=FALSE,
            reformat=FALSE)
    #print(paste0("print ",s," outputs into row ",start_row))
    selected_row <- selected_row+1
}

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Predictions"

Editing "Daily Product Breakdown Forecast -WIP"

Writing to sheet "Forecast Outputs - Predictions"



ERROR: Error: Client error: (400) INVALID_ARGUMENT
* Client specified an invalid argument. Check error message and error details for more information.
* Invalid requests[0].updateCells: Attempting to write column: 29, beyond the last requested column of: 28


In [59]:
forecasted_rugs

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-0.002550821,-0.04215138,-0.03074042,-0.04636472,-0.04296767,-0.01315473,-0.02309507,-0.001790151,-0.007868658,0.003066594,⋯,-0.02251737,-0.004935833,-0.01355857,-0.01702067,-0.02701756,-0.01868919,-0.02082899,-0.02787883,-0.01087876,-0.02135611
